In [3]:
import polars as pl
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load the dataset
train_data = pl.read_parquet('/home/jupyter/data/jane-street/train.parquet')

# Display the first few rows (use .head() in an interactive notebook)
train_data.head()


date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,…,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,partition_id
i16,i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i64
0,0,1,3.889038,null,null,null,null,null,0.851033,0.242971,0.2634,-0.891687,11,7,76,-0.883028,0.003067,-0.744703,null,-0.169586,null,-1.335938,-1.707803,0.91013,null,1.636431,1.522133,-1.551398,-0.229627,null,null,1.378301,-0.283712,0.123196,null,null,…,null,null,-0.808103,null,-2.037683,0.727661,null,-0.989118,-0.345213,-1.36224,null,null,null,null,null,-1.251104,-0.110252,-0.491157,-1.02269,0.152241,-0.659864,null,null,-0.261412,-0.211486,-0.335556,-0.281498,0.738489,-0.069556,1.380875,2.005353,0.186018,1.218368,0.775981,0.346999,0.095504,0
0,0,7,1.370613,null,null,null,null,null,0.676961,0.151984,0.192465,-0.521729,11,7,76,-0.865307,-0.225629,-0.582163,null,0.317467,null,-1.250016,-1.682929,1.412757,null,0.520378,0.744132,-0.788658,0.641776,null,null,0.2272,0.580907,1.128879,null,null,…,null,null,-1.625862,null,-1.410017,1.063013,null,0.888355,0.467994,-1.36224,null,null,null,null,null,-1.065759,0.013322,-0.592855,-1.052685,-0.393726,-0.741603,null,null,-0.281207,-0.182894,-0.245565,-0.302441,2.965889,1.190077,-0.523998,3.849921,2.626981,5.0,0.703665,0.216683,0.778639,0
0,0,9,2.285698,null,null,null,null,null,1.056285,0.187227,0.249901,-0.77305,11,7,76,-0.675719,-0.199404,-0.586798,null,-0.814909,null,-1.296782,-2.040234,0.639589,null,1.597359,0.657514,-1.350148,0.364215,null,null,-0.017751,-0.317361,-0.122379,null,null,…,null,null,-0.72542,null,-2.29417,1.764551,null,-0.120789,-0.063458,-1.36224,null,null,null,null,null,-0.882604,-0.072482,-0.617934,-0.86323,-0.241892,-0.709919,null,null,0.377131,0.300724,-0.106842,-0.096792,-0.864488,-0.280303,-0.326697,0.375781,1.271291,0.099793,2.109352,0.670881,0.772828,0
0,0,10,0.690606,null,null,null,null,null,1.139366,0.273328,0.306549,-1.262223,42,5,150,-0.694008,3.004091,0.114809,null,-0.251882,null,-1.902009,-0.979447,0.241165,null,-0.392359,-0.224699,-2.129397,-0.855287,null,null,0.404142,-0.578156,0.105702,null,null,…,null,null,1.313203,null,-0.810125,2.939022,null,3.988801,1.834661,-1.36224,null,null,null,null,null,-0.697595,1.074309,-0.206929,-0.530602,4.765215,0.571554,null,null,-0.226891,-0.251412,-0.215522,-0.296244,0.408499,0.223992,2.294888,1.097444,1.225872,1.225376,1.114137,0.775199,-1.379516,0
0,0,14,0.44057,null,null,null,null,null,0.9552,0.262404,0.344457,-0.613813,44,3,16,-0.947351,-0.030018,-0.502379,null,0.646086,null,-1.844685,-1.58656,-0.182024,null,-0.969949,-0.673813,-1.282132,-1.399894,null,null,0.043815,-0.320225,-0.031713,null,null,…,null,null,0.476195,null,-0.771732,2.843421,null,1.379815,0.411827,-1.36224,null,null,null,null,null,-0.948601,-0.136814,-0.447704,-1.141761,0.099631,-0.661928,null,null,3.678076,2.793581,2.61825,3.418133,-0.373387,-0.502764,-0.348021,-3.928148,-1.591366,-5.0,-3.57282,-1.089123,-5.0,0


In [4]:
# Filter out the first 85 days
train_data = train_data.filter(pl.col("date_id") >= 85)

# Fill NaN values with 0 (or other preferred method)
train_data = train_data.fill_null(0)


In [5]:
# Define columns to exclude
exclude_columns = ['date_id', 'time_id', 'symbol_id', 'weight', 'partition_id'] + \
                  [col for col in train_data.columns if col.startswith('responder_')]

# Select numerical columns
numerical_columns = [col for col in train_data.columns if col not in exclude_columns]


In [6]:
# Convert Polars DataFrame to NumPy for scaling
scaler = MinMaxScaler()
numerical_data = train_data.select(numerical_columns).to_numpy()

# Scale numerical columns
scaled_data = scaler.fit_transform(numerical_data)

# Replace scaled data back in the Polars DataFrame
scaled_df = pl.DataFrame(scaled_data, schema=numerical_columns)
train_data = train_data.drop(numerical_columns).hstack(scaled_df)


In [8]:
train_data.head()

date_id,time_id,symbol_id,weight,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,partition_id,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,…,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78
i16,i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
85,0,0,3.109312,0.62552,-0.310852,-0.049727,0.534198,0.198891,-1.069264,0.08607,0.273769,-1.012277,0,0.472176,0.477115,0.46872,0.45553,0.484776,0.407683,0.374309,0.394273,0.417295,0.1125,0.545455,0.126415,0.004958,0.002516,0.004336,0.014975,0.011851,0.016182,0.240309,0.390066,0.529444,0.003447,0.45861,…,0.509989,0.686283,0.542306,0.523023,0.531473,0.055818,0.374445,0.479017,0.556938,0.308719,0.549029,0.430893,0.410758,0.452865,0.230994,0.586376,0.434765,0.368225,0.441863,0.424024,0.002338,0.005114,0.005936,0.474298,0.510294,0.007815,0.002883,0.005926,0.006122,0.000856,0.003352,0.033741,0.026014,0.027316,0.020704,0.026509,0.02228
85,0,1,4.173016,0.503355,-0.094774,1.376147,0.487139,-0.026323,1.262791,0.233934,0.022418,0.401515,0,0.472176,0.477115,0.46872,0.45553,0.484776,0.405874,0.371698,0.394531,0.417012,0.1125,0.545455,0.126415,0.006245,0.00335,0.005714,0.014975,0.013263,0.016182,0.36588,0.367337,0.638666,0.003447,0.74606,…,0.509989,0.663297,0.542306,0.437079,0.751671,0.045415,0.377243,0.468906,0.556938,0.483197,0.549029,0.430893,0.315496,0.452865,0.285042,0.535432,0.434765,0.371859,0.438014,0.424024,0.00409,0.006476,0.014503,0.384289,0.417708,0.009524,0.003674,0.008467,0.006049,0.001049,0.003284,0.033741,0.026014,0.027652,0.020568,0.026359,0.022217
85,0,2,1.381817,5.0,5.0,-1.982797,5.0,5.0,-0.205135,0.4033,0.793304,2.807026,0,0.472176,0.477115,0.46872,0.45553,0.484776,0.402329,0.368157,0.38335,0.416891,0.9875,0.090909,0.09434,0.008581,0.016353,0.017169,0.014975,0.013123,0.016182,0.312509,0.340699,0.396479,0.003447,0.333757,…,0.509989,0.823772,0.542306,0.523023,0.531473,0.049142,0.394022,0.489217,0.556938,0.611577,0.549029,0.430893,0.456492,0.452865,0.299704,0.565893,0.434765,0.371202,0.439432,0.424024,0.002338,0.005114,0.005936,0.474298,0.510294,0.012743,0.019702,0.02084,0.007583,0.006656,0.011351,0.033741,0.026014,0.027756,0.020197,0.026952,0.022362
85,0,7,2.106224,-0.781161,-0.631942,0.152099,-2.374984,-1.785865,-0.273755,-1.696202,-0.548582,-0.81586,0,0.472176,0.477115,0.46872,0.45553,0.484776,0.398757,0.374288,0.390192,0.417201,0.1125,0.545455,0.126415,0.003254,0.002536,0.003796,0.014975,0.026587,0.016182,0.297364,0.372744,0.772192,0.003447,0.413171,…,0.509989,0.223054,0.542306,0.523023,0.531473,0.044446,0.376744,0.471335,0.556938,0.6372,0.549029,0.430893,0.285015,0.452865,0.377153,0.589548,0.434765,0.370797,0.43708,0.424024,0.002338,0.005114,0.005936,0.474298,0.510294,0.006725,0.003575,0.005227,0.005233,0.000697,0.001762,0.033741,0.026014,0.027337,0.020534,0.026126,0.022389
85,0,8,1.707147,4.280499,4.502849,1.735608,4.230651,5.0,-0.107674,1.533022,5.0,-4.17551,0,0.472176,0.477115,0.46872,0.45553,0.484776,0.406411,0.374718,0.398336,0.417527,0.1625,0.0,0.1,0.021755,0.049965,0.050826,0.014975,0.019111,0.016182,0.280273,0.224878,0.41004,0.003447,0.34003,…,0.509989,0.785653,0.542306,0.523023,0.531473,0.044133,0.383788,0.464249,0.556

In [10]:
# Save the cleaned data as Parquet
train_data.write_parquet('/home/jupyter/data/RealEncoder.parquet', compression='snappy')



In [1]:
print("Columns in saved data:", train_data.columns)

NameError: name 'train_data' is not defined